[View in Colaboratory](https://colab.research.google.com/github/aksh98/Attribute_Classification/blob/master/BTP_10thApr.ipynb)

In [1]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 
!pip3 install torch
!pip3 install torchvision


In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpjcmop_x5/pubring.gpg' created
gpg: /tmp/tmpjcmop_x5/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


KeyboardInterrupt: ignored

In [3]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [5]:
# ========================================================

# State of the art Accuracy - 91.25

# ========================================================


import numpy as np
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import scipy.io
import cv2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import Normalize
import os
import pickle
from torchvision import datasets, transforms
import torchvision.utils as vutil
import argparse
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from argparse import ArgumentParser
from sklearn.model_selection import train_test_split
print("Done")


Done


In [6]:

%cd drive
%cd Colab Notebooks 
#/content/drive/Colab Notebooks
# !jar xvf btpaligned2.zip 
!ls

/content/drive
/content/drive/Colab Notebooks
BTP_10thApr.ipynb	      btpfinalattributes.csv  __MACOSX
BTP1.ipynb		      BTP.ipynb		      output1.txt
btpaligned2		      finallala.txt	      output22.txt
btpaligned2.zip		      full_attributes.txt     output2.txt
btpaligned.zip		      imgarr_3513.pkl	      output.txt
BTPattributefile.csv	      imgarr_3788.pkl	      Untitled0.ipynb
BTP_Attributes.ods	      IP.ipynb		      Untitled1.ipynb
btpattributesrefined.csv      lfw_aligned	      Untitled2.ipynb
btpattributesrefined.csv.ods  lfwcolorfaces.zip
btpdataset.txt		      lfw_crop_A


In [0]:
%cd drive
%cd Colab Notebooks
# !ls -l | wc -l
# %cd .
!ls

In [7]:

parser = argparse.ArgumentParser(description='Process some integers.')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()



usage: ipykernel_launcher.py [-h] [--batch-size N] [--test-batch-size N]
                             [--epochs N] [--lr LR] [--momentum M] [--no-cuda]
                             [--seed S] [--log-interval N]
ipykernel_launcher.py: error: unrecognized arguments: -f /content/.local/share/jupyter/runtime/kernel-ec9df0ab-0b8f-40de-9546-135bc18899bb.json


SystemExit: ignored

In [17]:
# kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
print ("her")
mini_batch_size = 64

transform = transforms.Compose([
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) ])

# train_loader = torch.utils.data.DataLoader('lfw_aligned', transform, batch_size = 64, shuffle=False)
# test_loader = torch.utils.data.DataLoader('lfw_aligned', transform, batch_size=64, shuffle=False)


      
def readcsv(filename):
    ifile = open(filename, "rU")
    reader = csv.reader(ifile, delimiter=",")
    rownum = 0
    a = []

    for row in reader:
        a.append (row)
        rownum += 1
    
    ifile.close()
    return a

# attributes = pd.read_csv('BTPattributefile.csv')
# n = 65
# print(len(attributes))
# attrbutes = []
# for i in range(0,13144):
#     img_name = attributes.iloc[i, 0]
#     landmarks = attributes.iloc[i, 2:].as_matrix()
#     attributes.append()

x = np.loadtxt('finallala.txt',delimiter = '\n',dtype=np.str)
print(len(x))
array = []
# h,w = 3513,41
h, w = 3788, 41
# h, w = 13143, 41 
matrix = [[0 for x in range(w)] for y in range(h)] 
images = []
# print(images)
print ("here")
#---------------------
for i in range(1,h):
    arr = x[i].split() # 76 = len(arr)
    name_len = len(arr)-41
# Adding name to matrix
    la = arr[0]+"_"
    for l in range(1,name_len-1):
        la += arr[l]+"_"
#     print(arr[name_len-1])
    if(float(arr[name_len-1]) > 99):
        la += "0" + arr[name_len-1]+".jpg"
    elif(float(arr[name_len-1]) > 9 and float(arr[name_len]) < 100):
        la += "00" + arr[name_len-1] +".jpg"    
    elif(float(arr[name_len-1]) < 10):
        la += "000" + arr[name_len-1] +".jpg"
#     matrix[i-1][0] = la
    images.append(la)
    
    for j in range(name_len-1,len(arr)-1):
#         if(float(arr[j+1]) > 0): 
#           matrix[i-1][j-name_len+1] = 1
        matrix[i-1][j-name_len+1] = arr[j+1]
#         else: 
#           matrix[i-1][j-name_len+1] = 0
#         insertion(arr,j+1)

# matrix = np.matrix(matrix)

print("Matrix size",len(matrix),len(matrix[0]))

# newmat = matrix.transpose()
# for i in range(0,41):
#     print(matrix[0][i],"-",i,"-")
# print("Matrix size",len(newmat),len(newmat[0]))

# print (matrix.shape)
# print (newmat.shape)
# mo = np.min(matrix,axis = 0)
# mo = np.min(newmat,axis = 1)

# print(mo)

# for i in range(0,41):
#     for j in range(0,3788):
#         normalized = ((newmat[i][j]-min(newmat[i]))/(max(newmat[i])-min((newmat[i]))))
# # print(normalized.shape)
# matrix = normalized.transpose()
# for i in range(0,41):
#     print(matrix[i][0],"-",i,"-")
    

her
13144
here
Matrix size 3788 1
(3788, 41)
(41, 3788)


TypeError: ignored

In [10]:
print (matrix.shape)

(3788, 41)


In [0]:
def insertion(arr,i):
    if(float(arr[i]) > 0):
        return 1
    else:
        return 0

def out(i):
    if(float(i) > 0):
        return 1
    else:
        return 0


In [0]:
img_arr = []
i = 0
for i in range(0,13143): # 523 in this case
    img = images[i]
    imgpath = os.path.join("lfw_aligned/",img)
    image = cv2.imread(imgpath)
  # print(np.shape(image))
    if(image is None):
        print("No Image loaded")
        print(img)
    else:
        img_arr.append(np.reshape(image,62500*3))
    mat = matrix[i][:]
    sample = {'image': image, 'att': mat}
    i = i+1
    if(i%100 == 0):
        print(i,end=" ")
# print(len(matrix[i]))
print(len(img_arr))

# img_arr(1,62500) and matrix(13143,74)


In [10]:
# print(len(img_arr), len(img_arr[0]))
# print(img_arr[2280][:])
# print(mydict)
f = open('imgarr_3788.pkl', 'rb')
mydict = pickle.load(f)
# mydict = pickle.load("imgarr_3788.pkl")
print(len(mydict))
# pickle.dump(img_arr,open("imgarr_3788.pkl","wb"))

3788


In [11]:
import sys
X_train, X_test, y_train, y_test = train_test_split( mydict, matrix, test_size=0.33, random_state=24)
print(len(X_train),len(X_train[0]))
print(len(y_train),len(y_train[0]))

2537 187500
2537 41


In [0]:
# input image size = 227*227 - 250*250
# ==============================================================
# ==============================================================

class InceptionB(nn.Module):
    def __init__(self,in_channels):
        super(InceptionB,self).__init__()
        # Around Hair
        self.fco_11 = nn.Linear(in_channels,512)
        self.fco_12 = nn.Linear(512,512)
        self.fco_13 = nn.Linear(512,13)

        # Facial Hair
        self.fco_21 = nn.Linear(in_channels,512)
        self.fco_22 = nn.Linear(512,512)
        self.fco_23 = nn.Linear(512,5)

        # Cheeks
        self.fco_31 = nn.Linear(in_channels,512)
        self.fco_32 = nn.Linear(512,512)
        self.fco_33 = nn.Linear(512,2)

        # Fat
        self.fco_41 = nn.Linear(in_channels,512)
        self.fco_42 = nn.Linear(512,512)
        self.fco_43 = nn.Linear(512,2)

        
    def forward(self,x):
        br1 = F.relu(self.fco_11(x))
        br1 = F.dropout(br1,p=0.5)
        br1 = F.relu(self.fco_12(br1))
        br1 = F.dropout(br1,p=0.5)
        br1 = self.fco_13(br1)
        
        br2 = F.relu(self.fco_21(x))
        br2 = F.dropout(br2,p=0.5)
        br2 = F.relu(self.fco_22(br2))
        br2 = F.dropout(br2,p=0.5)
        br2 = self.fco_23(br2)
        
        br3 = F.relu(self.fco_31(x))
        br3 = F.dropout(br3,p=0.5)
        br3 = F.relu(self.fco_32(br3))
        br3 = F.dropout(br3,p=0.5)
        br3 = self.fco_33(br3)
        
        br4 = F.relu(self.fco_41(x))
        br4 = F.dropout(br4,p=0.5)
        br4 = F.relu(self.fco_42(br4))
        br4 = F.dropout(br4,p=0.5)
        br4 = self.fco_43(br4)
        
        output = [br1,br2,br3,br4]
#         print("out ss- ",output)
#         print("len ss- ",len(output))

        return torch.cat(output,1)

# ==============================================================
# ==============================================================


class InceptionA(nn.Module):
    
    def __init__(self,in_channels):  
        super(InceptionA, self).__init__()
        channel = 4800
        # Gender
        self.conv3g = nn.Conv2d(in_channels, 300, kernel_size = 3)
        # relu - 5*5 pooling - 5*5 normalization
        self.fc1g = nn.Linear(channel,512)
        self.fc2g = nn.Linear(512,512)
        self.fc3g = nn.Linear(512,1)
        
        # Eyes 
        self.conv3e = nn.Conv2d(in_channels, 300, kernel_size = 3)
        self.fc1e = nn.Linear(channel,512)
        self.fc2e = nn.Linear(512,512)
        self.fc3e = nn.Linear(512,5)

        # Nose
        self.conv3n = nn.Conv2d(in_channels, 300, kernel_size = 3)
        self.fc1n = nn.Linear(channel,512)
        self.fc2n = nn.Linear(512,512)
        self.fc3n = nn.Linear(512,2)

        # Face
        self.conv3f = nn.Conv2d(in_channels, 300, kernel_size = 3)
        self.fc1f = nn.Linear(channel,512)
        self.fc2f = nn.Linear(512,512)
        self.fc3f = nn.Linear(512,6)

        # Mouth 
        self.conv3m = nn.Conv2d(in_channels, 300, kernel_size = 3)
        self.fc1m = nn.Linear(channel,512)
        self.fc2m = nn.Linear(512,512)
        self.fc3m = nn.Linear(512,4)

        # Others
        self.conv3o = nn.Conv2d(in_channels, 300, kernel_size = 3)
        self.others = InceptionB(channel)
        
        
    def forward(self,x):
        b1 = F.relu(self.conv3g(x))
        b1 = F.max_pool2d(b1, kernel_size=5)
        b1 = b1.view(1,4800)
        b1 = F.relu(self.fc1g(b1))
        b1 = F.dropout(b1,p=0.5)
        b1 = F.relu(self.fc2g(b1))
        b1 = F.dropout(b1,p=0.5)
        b1 = self.fc3g(b1)
        
        b2 = F.relu(self.conv3e(x))
        b2 = F.max_pool2d(b2, kernel_size=5)
        b2 = b2.view(1,4800)
        b2 = F.relu(self.fc1e(b2))
        b2 = F.dropout(b2,p=0.5)
        b2 = F.relu(self.fc2e(b2))
        b2 = F.dropout(b2,p=0.5)
        b2 = self.fc3e(b2)
        
        b3 = F.relu(self.conv3n(x))
        b3 = F.max_pool2d(b3, kernel_size=5)
        b3 = b3.view(1,4800)
        b3 = F.relu(self.fc1n(b3))
        b3 = F.dropout(b3,p=0.5)
        b3 = F.relu(self.fc2n(b3))
        b3 = F.dropout(b3,p=0.5)
        b3 = self.fc3n(b3)
        
        b4 = F.relu(self.conv3f(x))
        b4 = F.max_pool2d(b4, kernel_size=5)
        b4 = b4.view(1,4800)
        b4 = F.relu(self.fc1f(b4))
        b4 = F.dropout(b4,p=0.5)
        b4 = F.relu(self.fc2f(b4))
        b4 = F.dropout(b4,p=0.5)
        b4 = self.fc3f(b4)
        
        b5 = F.relu(self.conv3m(x))
        b5 = F.max_pool2d(b5, kernel_size=5)
        b5 = b5.view(1,4800)
        b5 = F.relu(self.fc1m(b5))
        b5 = F.dropout(b5,p=0.5)
        b5 = F.relu(self.fc2m(b5))
        b5 = F.dropout(b5,p=0.5)
        b5 = self.fc3m(b5)
        
        b6 = F.relu(self.conv3o(x))
        b6 = F.max_pool2d(b6, kernel_size=5)
        b6 = b6.view(1,4800)
        b6 = self.others(b6)
        
        outputs = [b1,b2,b3,b4,b5,b6] # + b6
#         print("out - ",outputs)
#         print("len - ",len(outputs))
        
        return torch.cat(outputs,1)
        
# ==============================================================
# ==============================================================


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # relu - 3*3 max pooling - 5*5 normalization
        self.conv1 = nn.Conv2d(3, 75, kernel_size = 7)
        
        # relu - 3*3 pooling - 5*5 normalization
        self.conv2 = nn.Conv2d(75, 200, kernel_size = 5)
#         nn.BatchNorm2d(5*5)
        # inception in_channels = 200
        self.incept = InceptionA(in_channels = 200)
        self.fcaux = nn.Linear(40,40)
    
    def forward(self, x):
#         print(x.size(0))
        in_size = x.size(0)
        x = F.max_pool2d(F.relu(self.conv1(x)),(3,3))
        x = F.max_pool2d(F.relu(self.conv2(x)),(3,3))
        x = self.incept(x)
        x = self.fcaux(x)
#         print(x.shape())
        return x

file = open('output22.txt', 'w')
# sys.stdout = file
# net = Net()
# net.cuda
model = Net()
model = model.double()
print(model)
# file.write(model)
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.5)
criterion = nn.MSELoss()


def train(epoch):
    model.train()
    batch_idx = 0
#     print(len(X_train),len(X_train[0]),len(y_train))
    for (data, target) in zip(X_train,y_train):
        batch_idx = batch_idx+1
        target = torch.from_numpy(np.array(target,dtype='double')).double()
        data = torch.from_numpy(np.array(data,dtype='double')).double()
        data, target = Variable((data),requires_grad=True), Variable((target),requires_grad=True)
#         if args.cuda:
#             data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        data = data.view(1,3,250,250)
        output = model(data)
        i = 0
        loss_seq = []
        output = output.view(40,-1)
        out = output.data.numpy()
        tar = target.data.numpy()
#         out = (out > 0).astype(np.int_)
#         tar = (tar > 0).astype(np.int_)
        for o in out:
            if(i == 17):
#                 currloss = min(criterion(o,target[i]),criterion(o,target[i]))
#                 print("curr - ",currloss, " i - ", i, criterion(o),target[i]),criterion(o,target[i+1])
                currloss = min(abs((o-tar[i+1])*(o-tar[i+1])),abs((o-tar[i])*(o-tar[i])))
                i = i+2
            else:
#                 currloss = criterion(o,target[i])
#                 print("curr.. - ",currloss, " i.. - ", i )
                currloss = abs((o-tar[i])*(o-tar[i]))
                i = i+1
            loss_seq.append(currloss)
      
        loss = sum(loss_seq)/40
        loss = torch.from_numpy(loss)
        output = torch.from_numpy(out)
        target = torch.from_numpy(tar)
        loss = Variable(loss,requires_grad=True).double()
        output = Variable(output,requires_grad=True).double()
        target = Variable(target,requires_grad=True).double()
        loss.backward()
        optimizer.step()
        if(batch_idx % 100 == 0):
            file.write('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(y_train),
                100. * batch_idx / len(y_train), loss.data[0]))
            
            print('  Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}   '.format(
                epoch, batch_idx * len(data), len(y_train),
                100. * batch_idx / len(y_train), loss.data[0]),'\n')
            
def mse_loss(input, target):
    return torch.sum((input - target)**2) / input.data.nelement()
  
def test():
    model.eval()
    batch_idx = 0
    test_loss = 0
    correct = 0
    
    for data, target in zip(X_test,y_test):
        target = torch.from_numpy(np.array(target,dtype='double')).double()
        data = torch.from_numpy(np.array(data,dtype='double')).double()
        data, target = Variable(data, volatile=True), Variable(target,requires_grad=True)
        
        data = data.view(1,3,250,250)
        loss_seq = []
        output = model(data)
        i = 0
        temp = 0
        output = output.view(40,-1)
        out = output.data.numpy()
        tar = target.data.numpy()        
        # sum up batch loss
        corr = 0
        for o in out:
            if(i == 17):
#                 currloss = min(F.nll_loss(o,target(i+3)),F.nll_loss(o,target[i+2]))
#                   currloss = min(criterion(o,target[i]),criterion(o,target[i+1]))
                currloss = min(abs((o-tar[i+1])*(o-tar[i+1])),abs((o-tar[i])*(o-tar[i])))
                if(abs(o-tar[i+1]) < 0.6 or abs(o-tar[i])<0.6):
                    correct += 1
                i = i+2
      
            else:
#                 currloss = F.nll_loss(o,target[i+2])
#                 currloss = criterion(o,target[i])
                currloss = abs((o-tar[i])*(o-tar[i]))
                if((o-tar[i]) < 0.6):
                    correct += 1
                i = i+1
                
            loss_seq.append(currloss)
        test_loss += sum(currloss)/40
#         correct = correct + (corr/40)
        output = torch.from_numpy(out)
        target = torch.from_numpy(tar)
#         loss = Variable(loss,requires_grad=True).double()
        output = Variable(output,requires_grad=True).double()
        target = Variable(target,requires_grad=True).double()
        
#         test_loss = torch.nn.MSELoss(size_average=False)
        # get the index of the max log-probability
#         pred = output.data.max(1, keepdim=True)[1]
#         correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(y_test)
    print('\n Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%) \n'.format(test_loss, correct, len(y_test), 100. * correct / len(y_test)*40))
    file.write('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(y_test), 100. * correct / len(y_test)*40))

for epoch in range(1, 5):
    train(epoch)
    test()
file.close()

1 300 4 13 - br1 |  
1 300 4 5  - br2 |
1 300 4 2  - br3 |
1 300 4 2  - br4

br len = 4

---

1x300x4x1
1x300x4x5
1x300x4x2
1x300x4x6
1x300x4x4
1x300x4x22

In [0]:
ls